In [1]:
import pandas as pd
import numpy as np
from utils import CHARAS_LIST
import zipfile
from tqdm import tqdm
from joblib import delayed, Parallel
import warnings 
warnings.filterwarnings('ignore')

In [2]:
from models.PCA import PCA

In [3]:
with zipfile.ZipFile('data.zip', 'r') as z:  
    with z.open('data/datashare.pkl') as f:
        print('Reading datashare.pkl', end=' ')
        datashare = pd.read_pickle(f)
        datashare['DATE'].drop_duplicates().reset_index(drop=True).to_pickle('data/mon_list.pkl')
        # datashare.to_pickle('data/datashare.pkl')
        print('Done!')

Reading datashare.pkl Done!


In [18]:
def pre_process(date):
    cross_slice = datashare.loc[datashare.DATE == date].copy(deep=False)
    omitted_mask = 1.0 * np.isnan(cross_slice.loc[cross_slice['DATE'] == date])
    # fill nan values with each factors median
    cross_slice.loc[cross_slice.DATE == date] = cross_slice.fillna(0) + omitted_mask * cross_slice.median()
    # if all stocks' factor is nan, fill by zero
    cross_slice.loc[cross_slice.DATE == date] = cross_slice.fillna(0)

    re_df = []
    # rank normalization
    for col in CHARAS_LIST:
        series = cross_slice[col]
        de_duplicate_slice = pd.DataFrame(series.drop_duplicates().to_list(), columns=['chara'])
        series = pd.DataFrame(series.to_list(), columns=['chara'])
        
        de_duplicate_slice['sort_rank'] = de_duplicate_slice['chara'].argsort().argsort()
        rank = pd.merge(series, de_duplicate_slice, left_on='chara', right_on='chara', how='right')['sort_rank']
        # if all values are zero, the results will contain nan
        rank_normal = ((rank - rank.min())/(rank.max() - rank.min())*2 - 1)
        re_df.append(rank_normal)
    re_df = pd.DataFrame(re_df, index=CHARAS_LIST).T.fillna(0)
    re_df['permno'] = list(cross_slice['permno'].astype(int))
    re_df['DATE'] = list(cross_slice['DATE'].astype(int))
    
    return re_df[['permno', 'DATE'] + CHARAS_LIST]

In [11]:
tmp_df = pre_process(19570329)

In [19]:
processed_df = Parallel(n_jobs=-1)(delayed(pre_process)(d) for d in tqdm(datashare.DATE.drop_duplicates().to_list(), colour='green', desc='Processing'))

Processing: 100%|██████████| 718/718 [02:04<00:00,  5.76it/s]


In [20]:
processed_df = pd.concat(processed_df)

In [24]:
processed_df.reset_index(drop=True)

In [ ]:
rank_normal_series(datashare['acc']).hist()

In [ ]:
datashare_re = datashare[['permno', 'DATE']].copy(deep=False)

In [ ]:
for col in tqdm(CHARAS_LIST):
    datashare_re[col] = rank_normal_series(datashare[col])
    

In [ ]:
datashare_chara.loc[datashare_chara.DATE == 19670331]['acc'].drop_duplicates()

In [ ]:
datashare_chara['acc'].hist()